# Evaluate Datasets
Count how much data we have from each source, compare predictions and estimate runtimes. What I'll need to determine a suitable pipeline later.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import gzip

Configuration

In [ ]:
to_read = ['mdf-mos']  # Names of the molecule data sources
config_names = ['xtb', 'mopac_pm7', 'cp2k_b3lyp_svp', 'cp2k_b3lyp_tzvpd', 'cp2k_wb97x_d3_tzvpd']  # Approaches, ordered by increasing runtime
approx_names = ['vertical', 'acn-vertical', 'adiabatic', 'acn-adiabatic']  # Approximations for redox potential, increasing by runtime
redox_properties = ['oxidation_potential', 'reduction_potential']

## Load in the datasets
Load in the dataset and the time estimates for each step

In [ ]:
records = {}

Start by loading molecule records with the properties and identity

In [ ]:
properties = set()
recipes = set()
for source in to_read:
    with gzip.open(f'datasets/{source}.json.gz') as fp:
        for line in tqdm(fp, desc=source):
            data = json.loads(line)
            
            # Skip if there are no conformers, 
            #  which means no computations succeeded
            if len(data['conformers']) == 0:
                continue
            
            # Start by storing the identity and source
            record = defaultdict(float)
            record.update({
                'key': data['key'],
                'source': source,
                'smiles': data['identifier']['smiles'],
                'n_atoms': int(data['conformers'][0]['xyz'].split("\n")[0])
            })
            
            # Store the properties
            for prop, values in data['properties'].items():
                properties.add(prop)
                for level, value in values.items():
                    if abs(value) > 20 and 'potential' in prop:  # Some QM9 computations go off
                        #print(f'Failure of {data["key"]} for {prop}//{level}')
                        continue
                    recipe = f'{prop}.{level}'
                    recipes.add(recipe)
                    record[recipe] = value
            
            # Add to the total record list
            records[record['key']] = record

Load in the runtimes

In [ ]:
for source in to_read:
    try:
        fp = gzip.open(f'datasets/{source}-results.json.gz')
        for line in tqdm(fp, desc=source):
            data = json.loads(line)
            
            # Skip if unsuccessful, or if source does not not match
            key = data["task_info"]["key"]
            if not data['success'] or key not in records:
                continue
            record = records[key]
            if record['source'] != source:
                continue
                
            # Update the times
            prop = data["task_info"]["recipe"]
            level = data["task_info"]["level"]
            tag = f'runtime.{prop}.{level}'
            record[tag] += data["time_running"]
            
            # If the type is a neutral relaxation, add it the other potential
            if data["method"] == "optimize_structure" and level.endswith("vertical"):
                prop = "oxidation_potential" if prop == "reduction_potential" else "reduction_potential"
                tag = f'runtime.{prop}.{level}'
                record[tag] += data["time_running"]
    except EOFError:  # Thrown if gzip file is still being written
        continue
    finally:
        fp.close()

Convert it to a dataframe

In [ ]:
records = pd.DataFrame(records.values())

In [ ]:
print(f'Loaded a dataset of {len(records)} molecules with {len(properties)} properties computed at a total of {len(recipes)} levels')

CP2K tasks are checkpointed, so the runtimes are probably longer than reported

In [ ]:
recipes

## Compute Cumulative Runtime
Assume that we execute recipes by doing all approximations with the cheapest configuration before progressing to the next configuration.

In [ ]:
run_order = [
    f'{c}-{a}'
    for c in config_names
    for a in approx_names
]
print(f'Generated {len(run_order)} recipes:')
run_order

Compute the cumulative runtimes

In [ ]:
for prop in redox_properties:
    each_levels = [f'runtime.{prop}.{l}' for l in run_order]
    total_levels = [f'cumulative.{prop}.{l}' for l in run_order]
    
    records[total_levels] = records[each_levels].cumsum(axis=1)

Notes:
- 12Sep23: I switched from 4 workers per node on 1, and also from temporary files on global filesystems to node-level shm. 
- 20Sep23: Use FIRE for relaxation first
- 02Oct23: Switched from BFGSLineSearch to BFGS
- 24Jan23: Increased NBUFFER and number of electronic steps

## Compute Error
Compute the error at each level wrt to the top level of accuracy. We are going to use MAE after correcting for any offsets between the two methods

In [ ]:
print(f'Computing error relative to: {run_order[-1]}')

In [ ]:
fig, axxs = plt.subplots(len(redox_properties), len(run_order) - 1, figsize=(2.5 * (len(run_order) - 1), 5.), sharey='row', sharex='row')

for prop, axs in zip(redox_properties, axxs):
    target_col = f'{prop}.{run_order[-1]}'
    if target_col not in records.columns:
        print(f'No data for {target_col}')
        continue
    count = np.logical_not(records[target_col].isnull()).sum()
    print(f'Found {count} records for {prop}')
    for level, ax in zip(run_order, axs):
        col = f'{prop}.{level}'
        if col not in records.columns:
            print(f'No data for {col}')
            continue
        
        # Subtract off the average error
        mae = (records[col] - records[target_col]).abs().mean()
        mean_error = (records[col] - records[target_col]).mean()
        mae_corrected = (records[col] - mean_error - records[target_col]).abs().mean()
        print(f'Prop: {prop} - Level: {level} - MAE: {mae:.2f} - MAE,corrected: {mae_corrected:.2f}')
        
        ax.scatter(records[col] - mean_error, records[target_col], s=4, ec='none', alpha=0.7)
        
    # Format the axes
    axs[0].set_ylabel(f'{prop} (V)\n{config_names[-1]}\n{approx_names[-1]}', fontsize=8)
    
# Add y=x
for ax in axxs.flatten():
    ax.set_xlim(ax.get_ylim())
    ax.set_ylim(ax.get_ylim())
    ax.plot(ax.get_xlim(), ax.get_xlim(), 'k--')
    ax.set_xticklabels([])
    
# Add the level titles
for ax, level in zip(axxs[0, :], run_order):
    for name in config_names:
        if level.startswith(name):
            conf = name
            appr = level[len(conf)+1:]
    ax.set_title(f'{conf}\n{appr}', fontsize=8)
fig.tight_layout()
fig.savefig('figures/pred-vs-actual-redox.png', dpi=320)

Create a Pareto plot which shows the cumulative runtime vs accuracy

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6.5, 2.), sharex=True, sharey=True)

summary = []
for prop, ax in zip(redox_properties, axs):
    # Get the levels which were ran
    levels = [f'{prop}.{l}' for l in run_order if f'{prop}.{l}' in records.columns]
    print(f'Comparing {prop} to {levels[-1]}')
    
    # Compute the MAE without the offest
    error_all = (records[levels].values - records[levels[-1]].values[:, None])
    offset_all = np.nanmedian(error_all, axis=0)
    mae_by_level = np.nanmedian(np.abs(error_all - offset_all), axis=0)
    best_so_far = [min(mae_by_level[:i + 1]) for i in range(len(mae_by_level))]
    is_best = np.isclose(mae_by_level, best_so_far)
    
    # Compute the runtime for complete computations
    runtime_all = records[[f'cumulative.{l}' for l in levels]].values
    runtime_all[np.isnan(error_all)] = np.nan
    runtime_by_level = np.nanmedian(runtime_all, axis=0)
    
    ax.scatter(runtime_by_level, mae_by_level, 
               color=['seagreen' if i else 'lightsalmon' for i in is_best])
    
    summary.append(pd.DataFrame({
        'property': [prop] * len(levels),
        'level': [l[len(prop) + 1:] for l in levels],
        'count': np.isfinite(error_all).sum(axis=0),
        'runtime': runtime_by_level,
        'improved': is_best,
        'error': mae_by_level,
    }))

    # Prettify
    ax.set_xlabel('Runtime (s)')
    ax.set_title(prop, fontsize=10)
    ax.set_xscale('log')

axs[0].set_ylabel('MAE (V)')
summary = pd.concat(summary)

fig.tight_layout()
fig.savefig('figures/recipe-pareto-plot-redox.png', dpi=320)

In [ ]:
summary.to_csv('runtime-vs-performance-summary.csv', index=False)

Save a copy of the dataset with the columns in a reasonable order

In [ ]:
sorted_cols = sorted(records.columns, key=lambda x: (x.count("."), x))
print(f'Displaying columns: {",".join(sorted_cols[:6])},...')

In [ ]:
records[sorted_cols].to_csv(f'datasets/{"-".join(to_read)}_consolidated.csv.gz', index=False)